In [1]:
! python -m pip install gurobipy
! python -m pip install pandas


[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: pip install --upgrade pip


In [ ]:
import pandas as pd
import gurobipy as gp
from gurobipy import GRB

# Lettura file CSV
df = pd.read_csv("data/cup.csv")
idx = df.columns.tolist()[0]
df = df.set_index(f"{idx}")


# Definizione degli insiemi

## I: insieme delle località
I = df.drop(columns="domanda").columns.tolist()

## J: insieme dei quartieri
J = df.drop(index="costo").index.tolist()

# Definizione dei parametri

## Costo apertura CUP nella località i
C = df.loc["costo"].to_dict()

## Domanda di servizi CUP del quartiere j
D = df["domanda"].to_dict()

## Matrice A, A_ij: tempo di spostamento dal quartiere j alla località i
A = df.drop(index="costo").drop(columns="domanda")

print("Località (I):", I)
print("Quartieri (J):", J)
print("Costi (C):", C)
print("Domande (D):", D)
print("Coeff. (A):\n", A)


# Creazione modello
m = gp.Model("modello_dieta_cup")

# Creazione variabili decisionali
x = m.addVars(I, vtype=GRB.BINARY, lb=0, name="x")

V = ["q1", "q2"]

# Creazione vincoli
for j in J:
    sum = gp.quicksum(A.loc[j, i] * x[i] for i in I)

    copertura = D[j]
    if j in V:
        copertura = 2

    m.addConstr(
        sum >= copertura,
        name=f"resource_{j}",
    )

# Definizione funzione obiettivo
m.setObjective(gp.quicksum(C[i] * x[i] for i in I), GRB.MINIMIZE)

# Ottimizza
m.optimize()

# Stampa risultati
if m.status == GRB.OPTIMAL:
    print(f"Obiettivo (numero di CUP aperti): {m.objVal:.0f}")
    for i in I:
        val = int(x[i].X)
        if val > 0:
            print(f"CUP x[{i}] aperto")
        else:
            print(f"CUP x[{i}] chiuso")

elif m.status == GRB.INFEASIBLE:
    print("Modello inammissibile: prova ad aumentare T o controlla i dati del CSV.")
else:
    print(f"Stato ottimizzazione: {m.status}")

Località (I): ['loc1', 'loc2', 'loc3', 'loc4', 'loc5', 'loc6']
Quartieri (J): ['1', '2', '3', '4', '5', '6']
Costi (C): {'loc1': 1.0, 'loc2': 1.0, 'loc3': 1.0, 'loc4': 1.0, 'loc5': 1.0, 'loc6': 1.0, 'domanda': nan}
Domande (D): {'1': 1.0, '2': 1.0, '3': 1.0, '4': 1.0, '5': 1.0, '6': 1.0, 'costo': nan}
Coeff. (A):
            loc1  loc2  loc3  loc4  loc5  loc6
quartiere                                    
1             1     1     0     0     0     0
2             1     1     0     0     0     1
3             0     0     1     1     0     0
4             0     0     1     1     0     0
5             0     0     0     1     1     0
6             0     1     0     0     1     1
Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (mac64[arm] - Darwin 24.6.0 24G90)

CPU model: Apple M2 Pro
Thread count: 10 physical cores, 10 logical processors, using up to 10 threads

Optimize a model with 6 rows, 6 columns and 14 nonzeros
Model fingerprint: 0x486db3e9
Variable types: 0 continuous, 6 integer (